In [2]:
import psycopg2
import pandas as pd
import numpy as np
import glob
import random
import glob
import os
import openpyxl
import re
from fuzzywuzzy import fuzz
from collections import defaultdict

# Project Workflow: User Performance Analysis

### Data Enginerring
    1. Some data will be in the spreadhseets because they were not all updated into my postgres instance. The remaining will be in my postgres instance.
        * Extract spreadsheet data with existing functions and calls and use psycopg2 to extract data from postgres instance. Combine into a single Pandas dataframe.

### Data Analysis
    1. Create some individual user dashboards and one large dashboard
    2. Visualize elements like volume load over time, most frequent exercises prescribed, most frequent exercises skipped, how many workouts (from postgres) were completed, how many workouts were skipped. 
    3. Comparison between loaded and bodywieght exercises

In [3]:
#Extracting prescription data from database

conn=psycopg2.connect(dbname='cabral_fitness',
user='postgres',
password="Maninthemirror1995",
host='34.102.107.137',
port='5432',
sslmode='disable')

cursor=conn.cursor()

#For business analysis
cursor.execute("""SELECT c.name AS NAME, 
workout_number, block_id, b.workouts_per_week, 
e.exercise AS exercise_name, sets, reps, p.weight 
FROM prescriptions p 
LEFT JOIN client c on p.client_id=c.id
LEFT JOIN exercises e on p.exercise_id=e.id
LEFT JOIN blocks b on p.block_id=b.id
WHERE c.name not in  ('Test','Jared')
ORDER BY c.name, block_id, workout_number;
""")
db_prescriptions=cursor.fetchall()
conn.close()

In [4]:
db_prescriptions=pd.DataFrame(db_prescriptions, columns=['name','workout_number','block_id','workouts_per_week','exercise_name','sets','reps','weight'])

In [5]:
db_prescriptions

,name,workout_number,block_id,workouts_per_week,exercise_name,sets,reps,weight
0,Abelev,0,14,2,Banded Walks,3,60,1
1,Abelev,0,14,2,Split Squat Jumps,3,10,1
2,Abelev,0,14,2,Banded Squats,3,10,1
3,Abelev,0,14,2,RFE Lunges,3,10,1
4,Abelev,0,14,2,Side Planks,3,30,1
...,...,...,...,...,...,...,...,...
2843,Yelena,15,48,4,Z-Press,3,10,5
2844,Yelena,15,48,4,Curl to OH Press,3,10,3
2845,Yelena,15,48,4,Pushups,3,10,1
2846,Yelena,15,48,4,Bicyles,3,30,1


In [6]:
%store -r all_formatted_data
%store -r names

all_formatted_data=all_formatted_data
names=names

In [7]:
char_to_remove = [str(i) for i in range(10)]
specials_to_remove = ['*', '/', '**', '***', '(', '[', ')', ']', '&']
yelenas_to_remove = ['did', 'doing', 'didnt', 'need', 'could', 'couldnt', '?']
emphasis_to_remove = ['emphasis', 'Emphasis', 'Focus', 'Condition', 'recon', 'recondition', 'Recondition', 'wedding']

def passes_filters(exercise):
    if any(char in exercise for char in char_to_remove):
        return False
    if any(char in exercise for char in specials_to_remove):
        return False
    if any(char in exercise for char in yelenas_to_remove):
        return False
    if any(char in exercise for char in emphasis_to_remove):
        return False
    return True

In [8]:
def convert_to_int(data):
    try:
        for item in data:
            item['weight'] = int(float(item['weight']))
            item['sets'] = int(float(item['sets']))
            item['reps'] = int(float(item['reps']))
    except ValueError:
        return None

    return data
            

In [9]:
cleaned_dict={}
for client in all_formatted_data:
    client_data=[]
    for workout in all_formatted_data[client]:
        for rows in all_formatted_data[client][workout]:
            client_data.append(rows)
    cleaned_dict[client]=client_data

In [10]:
all_formatted_data={}
for client in cleaned_dict:
    flitered_client=[]
    try:
        for item in cleaned_dict[client]:
            if passes_filters(item['exercise']):
                flitered_client.append(item)
    except:
        pass
    all_formatted_data[client]=flitered_client

In [11]:
for client in all_formatted_data:
    converted_data = convert_to_int(all_formatted_data[client])
    all_formatted_data[client] = converted_data

In [12]:
all_formatted_data.keys()

dict_keys(['Dimitri', 'Masha', 'Misha', 'Anya', 'Arne', 'Max', 'Dino', 'Test', 'Ilya', 'Igor', 'Nick_Mary', 'Daniel Murphy', 'Yelena', 'Shawn'])

In [13]:
all_formatted_data.pop('Test', None)

[]

In [14]:
old_key = 'Daniel Murphy'
new_key = 'Daniel'

if old_key in all_formatted_data:
    all_formatted_data[new_key] = all_formatted_data[old_key]
    del all_formatted_data[old_key]

In [15]:
old_key = 'Max'
new_key = 'Abelev'

if old_key in all_formatted_data:
    all_formatted_data[new_key] = all_formatted_data[old_key]
    del all_formatted_data[old_key]

In [17]:
dfs=[]
for client in all_formatted_data:
    df=pd.DataFrame(all_formatted_data[client])
    dfs.append(df)

df=pd.concat(dfs, axis=0)
    

In [18]:
df

,exercise,weight,sets,reps
0,Repeat Squat Jumps,1,3,10
1,Split Squat Jumps,1,2,10
2,Lateral Walks,1,3,1
3,Banded Goblet Squats,1,3,10
4,Goblet Lunges,1,3,10
...,...,...,...,...
499,Repeat Squat Jumps,1,3,10
500,Split Squat Jumps,1,3,10
501,Dead Bugs,1,3,10
502,Plank Circuit,1,3,10


In [19]:
db_prescriptions=db_prescriptions.drop(columns=['name', 'workout_number','block_id','workouts_per_week'])
db_prescriptions

,exercise_name,sets,reps,weight
0,Banded Walks,3,60,1
1,Split Squat Jumps,3,10,1
2,Banded Squats,3,10,1
3,RFE Lunges,3,10,1
4,Side Planks,3,30,1
...,...,...,...,...
2843,Z-Press,3,10,5
2844,Curl to OH Press,3,10,3
2845,Pushups,3,10,1
2846,Bicyles,3,30,1


In [20]:
#Reordering so can concatenatye with df
db_prescriptions=db_prescriptions.rename(columns={'exercise_name':'exercise'})
db_prescriptions=db_prescriptions[['exercise','weight','sets','reps']]


In [21]:
db_prescriptions=db_prescriptions.reset_index(drop=True)
df=df.reset_index(drop=True)
big_df=pd.concat([df, db_prescriptions], axis=0, ignore_index=True)
big_df


,exercise,weight,sets,reps
0,Repeat Squat Jumps,1,3,10
1,Split Squat Jumps,1,2,10
2,Lateral Walks,1,3,1
3,Banded Goblet Squats,1,3,10
4,Goblet Lunges,1,3,10
...,...,...,...,...
8857,Z-Press,5,3,10
8858,Curl to OH Press,3,3,10
8859,Pushups,1,3,10
8860,Bicyles,1,3,30


### Part 1: Determine most frequently prescribed exercises

In [22]:
exercises=big_df['exercise']
exercises=exercises.str.lower()
exercises=exercises.value_counts()
exercises

mountain climbers        468
burpees                  255
banded walks             250
j-jacks                  232
plank to pushup          177
                        ... 
standing clam shells       1
eccentric q iso lunge      1
squat holds                1
high knees                 1
side plank +press          1
Name: exercise, Length: 531, dtype: int64

In [23]:
exercises[exercises>100].to_csv('top_exercises.csv')

In [24]:
exercises[exercises>100]

mountain climbers    468
burpees              255
banded walks         250
j-jacks              232
plank to pushup      177
plank circuit        174
side planks          163
db bench press       158
split squat jumps    145
sa db row            142
dead bugs            139
squat jumps          131
pushups              129
sl glute bridges     119
rfe lunges           111
bicycles             109
rdl                  106
sa renegade row      101
Name: exercise, dtype: int64

because of typos and other issues I am trying to use a string similairty system to combine the counts of exercises that are not included but otherwsise the same

In [25]:
unique_values = exercises.index.to_list()
similarity_threshold = 80

# Create a dictionary where the keys are strings and the values are lists of strings
similar_strings = defaultdict(list)

assigned_values = set()

for i in range(len(unique_values)):
    for j in range(i + 1, len(unique_values)):
        if unique_values[j] in assigned_values:
            continue
        similarity = fuzz.ratio(unique_values[i], unique_values[j])
        
        if similarity >= similarity_threshold:
            similar_strings[unique_values[i]].append(unique_values[j])
            assigned_values.add(unique_values[j])


# Now similar_strings dictionary holds strings as keys and lists of similar strings as values


In [26]:
similar_strings

defaultdict(list,
            {'burpees': ['mc-burpees'],
             'plank to pushup': ['bosu plank to pushup'],
             'plank circuit': ['planks circuit',
              'p-plank circuit',
              'plank circuiy'],
             'side planks': ['side planks ', 'side plank'],
             'db bench press': ['db oh press',
              'bench press',
              'bb bench press',
              'sa db bench press',
              'sa bench press'],
             'split squat jumps': ['repeat squat jumps',
              'no flip split squat jumps',
              'spli squat jumps'],
             'sa db row': ['sa db row '],
             'dead bugs': ['yb dead bugs', 'ar dead bugs', 'deas bugs'],
             'squat jumps': ['squat jumps '],
             'pushups': ['pushup'],
             'sl glute bridges': ['ffe sl glute bridges',
              'sl glute bridge',
              'glute bridges',
              'ffe sl glute bridge'],
             'rfe lunges': ['rfe lunge',
 

### Data Sanitization
    1. The approach above is not perfect but it satisfies the general goal to consolidate the string exercises that are similar to each other. In most cases it does what I want, however, in some it combines it based on words that are not related to the exercise. See below
            'partner crunch holds': ['partner crunch hold'],
             'leg press machine': ['chest press machine',
              'oh press machine',
              'shoulder press machine'],

     Ultimately, as long as the top exercises are not being lost, that is what matters

In [27]:
# Create a new Series to hold the combined counts
combined_exercises = pd.Series(dtype='int64')

#established combined count variable
for key, similar_values in similar_strings.items():
    combined_count = exercises[key] if key in exercises else 0

    # Add the counts for the similar exercises and increment counter
    for value in similar_values:
        if value in exercises:
            combined_count += exercises[value]

    # Set the count for the key exercise in the new series to the combined count
    combined_exercises[key] = combined_count

    # Remove the counts for the similar exercises from the original series
    for value in similar_values:
        if value in exercises:
            exercises = exercises.drop(value)

# Combine the original and the new Series
exercises = pd.concat([exercises, combined_exercises])





In [28]:
combined_exercises.sort_values(ascending=False, inplace=True)
combined_exercises

db bench press      359
burpees             256
plank to pushup     184
side planks         181
plank circuit       180
                   ... 
incline oh press      1
sa oh press           1
db bench              1
bench press           1
db incline bench      1
Length: 142, dtype: int64

In [29]:
exercises.sort_values(ascending=False, inplace=True)

In [30]:
exercises.drop_duplicates(inplace=True)
exercises

mountain climbers                468
db bench press                   359
burpees                          256
burpees                          255
banded walks                     250
                                ... 
oh triceps                         5
back hyperextenstions              4
partner crunch holds               3
reverse lunge-->step ups           2
ext rotator and psoas release      1
Length: 106, dtype: int64

In [31]:
exercises[exercises>100]

mountain climbers    468
db bench press       359
burpees              256
burpees              255
banded walks         250
j-jacks              232
plank to pushup      184
side planks          181
plank circuit        180
plank to pushup      177
sl glute bridges     175
plank circuit        174
rfe lunges           168
sa renegade row      167
dead bugs            165
side planks          163
db bench press       158
split squat jumps    157
stretches            155
sa db row            147
split squat jumps    145
sa db row            142
dead bugs            139
lb warm up           133
squat jumps          131
pushups              129
sl glute bridges     119
bicycles             117
rfe lunges           111
goblet squats        110
bicycles             109
rdl                  106
sa renegade row      101
dtype: int64

### End part 1:

1. There does not really appear to be much of a difference when the string processing is included except for db bench press which may include a lot of different exercises, but also erroneously included 'db oh press.' Ultimately not a great system and sometimes I got different results for the same exercises

2. A better approach to deal with this on the front end would probably be to use regex and other simple rules like str.lower() before my data gets input into teh database. The old spreadhseet data I cannot (or will not) do this for, but for the new data I am going to add these simple rules to make sure typos like extra spaces before/after and capitilization dont influence the exercise

3. In the next section I will do something somewhat similar

# Part 2: Using Regex to create movement categories to determine most common focus for workouts

In [32]:
big_df['exercise']=big_df['exercise'].str.lower()

In [66]:
keywords=['dl', 'deadlift', '\\bthrust\\b', 'thurst', 'bridge', 'hamstring']
regex_pattern='|'.join(keywords)
lower_body_pull=big_df[big_df['exercise'].str.contains(regex_pattern)]
lower_body_pulls=lower_body_pull['exercise'].unique()
lower_body_pulls

array(['sl glute bridges', 'sl rdl', 'sl rdl row', 'rdl',
       'hamstring planks', 'sl bridge', 'sl glute bridge', 'bb rdl',
       'bb deadlifts', 'sl rdl ', 'deadlifts', 'cg sldl', 'cgsldl',
       'hamstring stretch', 'yb hamstring curls', 'db sl rdl',
       'ffe sl glute bridges', 'sl rdl rows', 'banded bridge abductions',
       'banded hip thrust', 'single leg rdl', 'side lying hip thrust',
       'ipsilateral sl rdl row', 'banded weighted bridge',
       'sumo deadlift', 'banded abduction out of bridge',
       'nordic hamstring curls', 'hip thrust', 'bb sl rdl',
       'hamstring curl machine', 'bb hip thrust', 'sl bridges',
       'deadlift', 'yb hamstring', 'hip thrust or goblet squat',
       'deadlift or rdl', 'db sldl', 'db rdl', 'db rdl ', 'db sl rdl ',
       'bb rdl ', 'sl hip thrust', 'ffe sl glute bridge',
       'sl rdl without weight>', 'hamstring and calf', 'hamstring plank',
       'eccentric rdl', 'db sl rdl row', 'glute bridges', 'cg sl rdl',
       'yoga bal

In [35]:
keywords=['squat', 'lunge', 'split', 'step', 'leg press']
regex_pattern='|'.join(keywords)
lower_body_pushes=big_df[big_df['exercise'].str.contains(regex_pattern)]
lower_body_pushes=lower_body_pushes['exercise'].unique()
lower_body_pushes

array(['repeat squat jumps', 'split squat jumps', 'banded goblet squats',
       'goblet lunges', 'sa squat thruster', 'squat thruster',
       'squat thrusters', 'static lunge', 'rfe lunge',
       'goblet pause squats', 'goblet static lunge', 'sa oh static lunge',
       'sa oh lunge', 'goblet squats', 'sa squat thrusters',
       'goblet squat', 'goblet reverse lunge', 'goblet rfe lunge',
       'sa oh forward lunge', 'sa oh walking lunge', 'db rfe lunge',
       'kang squat', 'squat jumps', 'alternating reverse lunges',
       'squat jumps ', 'sa oh alternating forward lunge',
       'spli squat jumps', 'walking lunges', 'bb squat', 'bb lunges',
       'lunge to step up', 'deep squats', 'lunge holds',
       'eccentric q iso lunge', 'squat holds', 'rfe lunge hold',
       'static lunges', 'bb static lunge', 'bb walking lunges',
       'bb back squat', 'rfe lunges', 'banded pause goblet squats',
       'db static lunge', 'db static lunges', 'bw rfe lunge',
       'db walking lunges'

In [36]:
keywords = ['\\-\\>', '\\(.*\\)']
regex_pattern='|'.join(keywords)
super_sets=big_df[big_df['exercise'].str.contains(regex_pattern)]
super_sets=super_sets['exercise'].unique()
super_sets

array(['side plank-->plank-->side plank', 'bunny hops->squat jumps',
       'mountain cliombers->squat jumps', 'bunny hops-->squat jumps',
       'reverse lunge-->step up', 'reverse lunge-->step ups',
       'reverse lunge-->high knee', 'squat jumps (sl land)'], dtype=object)

In [37]:
keywords=['hop', 'jump', 'land']
regex_pattern='|'.join(keywords)
plyos=big_df[big_df['exercise'].str.contains(regex_pattern)]
plyos=plyos['exercise'].unique()
plyos

array(['repeat squat jumps', 'split squat jumps', 'squat jumps',
       'squat jumps ', 'spli squat jumps', 'bunny hops->squat jumps',
       'mountain cliombers->squat jumps', 'bunny hops-->squat jumps',
       'landmine row', 'jumping jacks', 'no flip split squat jumps',
       'squat jumps (sl land)', 'broad jumps'], dtype=object)

In [53]:
include_keywords=['bench', 'push', 'press', 'oh', 'incline']
exclude_keywords=['lunge', 'squat', 'row', "i'y't", 'i, y, t','leg', 'prone', 'i,y,t', "t'"]

regex_exclude_pattern = '|'.join(exclude_keywords)
regex_include_pattern = '|'.join(include_keywords)

# First, filter out exercises that include any of the excluded keywords
exercises_filtered = big_df[~big_df['exercise'].str.contains(regex_exclude_pattern, case=False, flags=re.IGNORECASE)]

# Then, select exercises that include any of the included keywords
upper_body_pushes = exercises_filtered[exercises_filtered['exercise'].str.contains(regex_include_pattern, case=False, flags=re.IGNORECASE)]
upper_body_pushes = upper_body_pushes['exercise'].unique()

print(upper_body_pushes)
print(len(upper_body_pushes))  


['pushups' 'sa bench press' 'plank to pushup' 'bench press'
 'hk sa db oh press' 'spider pushups' 'sa db push press'
 'side plank + press' 'sa db bench' 'sa db oh press' 'sa push press'
 'chest press machine' 'db bench press' 'sa oh press' 'plyo pushups'
 'deficit pushups' 'db incline bench' 'oh situps' 'bb bench'
 'pushups from knees' 'sa oh triceps extension' 'db bench' 'db oh press'
 'negative pushups' 'sa oh walks' 'bb oh press' 'bb bench press'
 'standing db oh press' 'sa hk db oh press' 'sa hk curl to oh press'
 'sa db bench press' 'trx pushup' 'curl to oh press' 'pushup'
 'trx chest press' 'z-press' 'pushup position arm raises' 'db push press'
 'seated db oh press' 'decline pushups' 'side plank +press'
 'db curl to oh press' 'sa oh db walks' 'tk db oh press'
 'oh press machine' 'incline oh press' 'oh triceps' 'trx pushups'
 'incline bench press' 'oh tricep extension' 'shoulder press machine'
 'sa oh tricep' 'db incline bench press' 'hk sa oh press'
 'kneeling sa oh press ' 'db o

In [59]:
include_keywords=['pull', 'row', 'down', 'point', 'renegade', 'chin', 'curl', 'snow', 'raise','prone', 'i,y,t']
exclude_keywords=['leg', 'yoga', 'hamstring', 'plank', 'calf']

regex_exclude_pattern = '|'.join(exclude_keywords)
regex_include_pattern = '|'.join(include_keywords)

# First, filter out exercises that include any of the excluded keywords
exercises_filtered = big_df[~big_df['exercise'].str.contains(regex_exclude_pattern, case=False, flags=re.IGNORECASE)]

upper_body_pulls=exercises_filtered[exercises_filtered['exercise'].str.contains(regex_include_pattern, case=False, flags=re.IGNORECASE)]

upper_body_pulls=upper_body_pulls['exercise'].unique()
upper_body_pulls

array(['renegade rows', 'sa db row', 'sl rdl row', 'curls',
       'sa lateral raise', 'sa snow angel', 'sa upright row',
       'prone oh press', 'incline seal rows', 'chest press machine',
       'lat pulldown machine', 'sa renegade row', "i,y,t's", 'seal rows',
       'db pullover', 'prone snow angels', 'lat pulldowns',
       'inclince renegade row', 'trx row', 'db seal rows',
       'standing banded row', 'standing banded high row',
       'lateral raises', 'standing snow angels', 'bent over row',
       'renegade row', 'incline seal row', 'banded face pulls',
       'db incline seal row', 'db bent over row', 'bb supinated row',
       'rower sprints', 'seated rows', 'inverted rows', 'db pullover ',
       'sa hk curl to oh press', 'stansing snow angels',
       'upward downward dog', 'trx high row',
       'bilateral db bent over row', 'curl to oh press', 'bicep curls',
       'trx bicep curls', 'strict elbow curls', 'snow angels',
       'full curls', 'pushup position arm raises

In [63]:
include_keywords=['plank', 'bug', 'banded', 'hold', '3:45', 'bicyle', 'pallof', 'twist', 'rotation', 'walks', 'crunch', 'situp']
exclude_keywords=['squat', 'lunge', 'standing']

regex_exclude_pattern = '|'.join(exclude_keywords)
regex_include_pattern = '|'.join(include_keywords)

# First, filter out exercises that include any of the excluded keywords
exercises_filtered = big_df[~big_df['exercise'].str.contains(regex_exclude_pattern, case=False, flags=re.IGNORECASE)]

core=exercises_filtered[exercises_filtered['exercise'].str.contains(regex_include_pattern, case=False, flags=re.IGNORECASE)]

core=core['exercise'].unique()
print(core)
print(len(core))


['lateral walks' 'plank to pushup' 'side planks ' 'oblique crunches'
 'ob crunches' 'side planks' 'side plank' 'side plank + press'
 'side plank + high knee' 'sa farmers walks' 'banded walks'
 'hamstring planks' 'situps' 'plank circuit' 'dead bugs'
 'side plank-->plank-->side plank' 'oh situps' 'yoga ball planks'
 "banded i, y, t's" 'banded clam shells' 'banded abductions' 'planks'
 'hollow holds' 'sa oh walks' 'ar dead bugs' 'banded face pulls'
 'side plank + row' 'partner crunch holds' 'yb dead bugs' 'deas bugs'
 'planks circuit' 'plank circuiy' 'side plank+abduction'
 'long lever plank' 'side plank +press' 'side planks from knees'
 'banded bridge abductions' 'banded hip thrust' 'russian twists'
 'lateral banded dead bugs' 'p-plank circuit' 'sa oh db walks'
 'banded weighted bridge' 'banded abduction out of bridge'
 'crunch machine' 'weighted oblique crunch' 'thai crunches'
 'lateral banded walks' 'plank pullbacks' 'partner crunch hold'
 'lateral banded walks ' 'yb situps' 'abduction

In [141]:
include_keywords=['stairs', 'mountain', 'burp', 'sprint', 'rower', 'bike', 'treadmill']
# exclude_keywords=['squat', 'lunge', 'standing']

# regex_exclude_pattern = '|'.join(exclude_keywords)
regex_include_pattern = '|'.join(include_keywords)

# First, filter out exercises that include any of the excluded keywords
cardio=big_df[big_df['exercise'].str.contains(regex_include_pattern)]
cardio=cardio['exercise'].unique()
print(cardio)
print(len(cardio))

['mountain climbers' 'burpees' 'bike sprints' 'sprints' 'hill sprints'
 'stairs' 'burpee-sprints' 'mc-burpees' 'rower sprints'
 'mountain cliombers->squat jumps' 'stairs ' 'bike']
12


In [142]:
movements=[upper_body_pushes, upper_body_pulls, lower_body_pulls, lower_body_pushes, plyos, super_sets, core, cardio]
movements

[array(['pushups', 'sa bench press', 'plank to pushup', 'bench press',
        'hk sa db oh press', 'spider pushups', 'sa db push press',
        'side plank + press', 'sa db bench', 'sa db oh press',
        'sa push press', 'chest press machine', 'db bench press',
        'sa oh press', 'plyo pushups', 'deficit pushups',
        'db incline bench', 'oh situps', 'bb bench', 'pushups from knees',
        'sa oh triceps extension', 'db bench', 'db oh press',
        'negative pushups', 'sa oh walks', 'bb oh press', 'bb bench press',
        'standing db oh press', 'sa hk db oh press',
        'sa hk curl to oh press', 'sa db bench press', 'trx pushup',
        'curl to oh press', 'pushup', 'trx chest press', 'z-press',
        'pushup position arm raises', 'db push press',
        'seated db oh press', 'decline pushups', 'side plank +press',
        'db curl to oh press', 'sa oh db walks', 'tk db oh press',
        'oh press machine', 'incline oh press', 'oh triceps',
        'trx pushu

### Quick postgres database Tangent

1. It is long overdue that i create movement tables with exercises that correspond to the movements. This will help with my AI model, with rule based prescriptions, and subseqeunt analysis. So quick tangent to do that

In [74]:
movements[0]

array(['pushups', 'sa bench press', 'plank to pushup', 'bench press',
       'hk sa db oh press', 'spider pushups', 'sa db push press',
       'side plank + press', 'sa db bench', 'sa db oh press',
       'sa push press', 'chest press machine', 'db bench press',
       'sa oh press', 'plyo pushups', 'deficit pushups',
       'db incline bench', 'oh situps', 'bb bench', 'pushups from knees',
       'sa oh triceps extension', 'db bench', 'db oh press',
       'negative pushups', 'sa oh walks', 'bb oh press', 'bb bench press',
       'standing db oh press', 'sa hk db oh press',
       'sa hk curl to oh press', 'sa db bench press', 'trx pushup',
       'curl to oh press', 'pushup', 'trx chest press', 'z-press',
       'pushup position arm raises', 'db push press',
       'seated db oh press', 'decline pushups', 'side plank +press',
       'db curl to oh press', 'sa oh db walks', 'tk db oh press',
       'oh press machine', 'incline oh press', 'oh triceps',
       'trx pushups', 'incline be

In [75]:
conn=psycopg2.connect(dbname='cabral_fitness',
user='postgres',
password="Maninthemirror1995",
host='34.102.107.137',
port='5432',
sslmode='disable')

cursor=conn.cursor()

table_names=['upper_body_pushes', 'upper_body_pulls', 'lower_body_pulls', 'lower_body_pushes', 'plyos', 'super_sets', 'core']

for table_name in table_names:
    cursor.execute(f"""CREATE TABLE {str(table_name)} 
    (id serial,
    exercise_id VARCHAR(100),
    PRIMARY KEY (id)
    );""")

conn.commit()

conn.close()




In [78]:
conn=psycopg2.connect(dbname='cabral_fitness',
user='postgres',
password="Maninthemirror1995",
host='34.102.107.137',
port='5432',
sslmode='disable')

cursor=conn.cursor()

for pattern, table_name in zip(movements,table_names):
    for exercise in pattern:
        cursor.execute("SELECT EXISTS(SELECT 1 FROM exercises WHERE exercise=%s);",(exercise,))
        exists = cursor.fetchone()[0]
        if not exists:
            cursor.execute("INSERT INTO exercises (exercise) VALUES (%s);", (exercise,))
        cursor.execute("SELECT id FROM exercises WHERE exercise = %s;", (exercise,))
        exercise_id = cursor.fetchone()[0]
        cursor.execute(f"""
        INSERT INTO {table_name} (exercise_id) VALUES (%s);
    """, (exercise,))

conn.commit()

conn.close()

### End postgres tangent

## To continue with analysis we have to use the new movement arrays that have been created through the regex rules to determine the most common focus for each workout. 

1. Generally in my practice I wrote the emphasis of a block at the top of a spreadsheet, since I have transitioned to postgres I have not been recording block 'emphasis' (which may be a good thing to include, another tangent for another day). 

2. Block emphasis != workout emphasis as usually workouts are split upper/lower or push/pull or body part specific if training for a specific element. 

3. This analysis will classify the clients' workouts based on the category that the majority of the exercises in each workout fall into."

In [92]:
#Grab the data that is still linked to each workout, we can do a sub male/female comparison

conn=psycopg2.connect(dbname='cabral_fitness',
user='postgres',
password="Maninthemirror1995",
host='34.102.107.137',
port='5432',
sslmode='disable')

cursor=conn.cursor()

#slight change from previous qeury to pull from workout_exercises instead of prescriptions
cursor.execute("""SELECT c.name AS NAME, 
workout_id,  
e.exercise AS exercise_name, sets, reps, w.weight 
FROM workout_exercises w 
LEFT JOIN client c on w.client_id=c.id
LEFT JOIN exercises e on w.exercise_id=e.id
WHERE c.name not in  ('Test','Jared')
ORDER BY c.name, workout_id;
""")
db_workouts=cursor.fetchall()
conn.close()


In [93]:
db_workouts=pd.DataFrame(db_workouts, columns=['name', 'workout_id', 'exercise', 'sets', 'reps', 'weight'])
db_workouts

,name,workout_id,exercise,sets,reps,weight
0,Abelev,60,Banded Squats,3,10,1
1,Abelev,60,RFE Lunges,3,10,1
2,Abelev,60,Side Planks,3,30,1
3,Abelev,60,Banded Walks,3,60,1
4,Abelev,60,Split Squat Jumps,3,10,1
...,...,...,...,...,...,...
1872,Yelena,306,RDL,4,5,65
1873,Yelena,306,BB Hip Thurst,4,5,65
1874,Yelena,306,Split Squat Jumps,3,5,1
1875,Yelena,306,Mountain Climbers,3,30,1


In [98]:
print(db_workouts['name'].unique())
print(all_formatted_data.keys())

['Abelev' 'Alan' 'Anya' 'Arne' 'Daniel' 'Dassi' 'Dimitri' 'Dino' 'Igor'
 'Ilya' 'Mary' 'Masha' 'Misha' 'Nat' 'Nick' 'Sachi' 'Shawn' 'Yelena']
dict_keys(['Dimitri', 'Masha', 'Misha', 'Anya', 'Arne', 'Dino', 'Test', 'Ilya', 'Igor', 'Yelena', 'Shawn', 'Daniel', 'Nick', 'Abelev'])


In [97]:
old_key = 'Daniel Murphy'
new_key = 'Daniel'

if old_key in all_formatted_data:
    all_formatted_data[new_key] = all_formatted_data[old_key]
    del all_formatted_data[old_key]

old_key = 'Nick_Mary'
new_key = 'Nick'

if old_key in all_formatted_data:
    all_formatted_data[new_key] = all_formatted_data[old_key]
    del all_formatted_data[old_key]

old_key = 'Max'
new_key = 'Abelev'

if old_key in all_formatted_data:
    all_formatted_data[new_key] = all_formatted_data[old_key]
    del all_formatted_data[old_key]

In [102]:
all_formatted_data['Dimitri']

{'workout_1': [{'exercise': 'Repeat Squat Jumps',
   'weight': 1,
   'sets': 3,
   'reps': 10},
  {'exercise': 'Split Squat Jumps', 'weight': 1, 'sets': 2, 'reps': 10},
  {'exercise': 'Lateral Walks', 'weight': 1, 'sets': 3, 'reps': 1},
  {'exercise': 'Banded Goblet Squats', 'weight': 1, 'sets': 3, 'reps': 10},
  {'exercise': 'Goblet Lunges', 'weight': 1, 'sets': 3, 'reps': 10},
  {'exercise': 'SL Glute bridges', 'weight': 1, 'sets': 3, 'reps': 10},
  {'exercise': 'Hip Flexor Stretch', 'weight': 1, 'sets': 1, 'reps': 1}],
 'workout_2': [{'exercise': 'J-jacks', 'weight': 1, 'sets': 1, 'reps': 1},
  {'exercise': 'UB Warm Up', 'weight': 1, 'sets': 2, 'reps': 1},
  {'exercise': 'Mountain Climbers', 'weight': 1, 'sets': 3, 'reps': 30},
  {'exercise': 'Pushups', 'weight': 1, 'sets': 4, 'reps': 8},
  {'exercise': 'SA Squat Thruster', 'weight': 20, 'sets': 4, 'reps': 8},
  {'exercise': 'SA Bench Press', 'weight': 35, 'sets': 4, 'reps': 8},
  {'exercise': 'Plank to Pushup', 'weight': 1, 'sets':

In [103]:
for client in all_formatted_data:
    new_formatted_data = []
    for workout_number, exercises in all_formatted_data[client].items():
        for exercise in exercises:
            exercise['workout_number'] = workout_number
        new_formatted_data.extend(exercises)
    all_formatted_data[client] = new_formatted_data


In [106]:
pd.DataFrame(all_formatted_data['Dimitri'])

,exercise,weight,sets,reps,workout_number
0,Repeat Squat Jumps,1.0,3,10,workout_1
1,Split Squat Jumps,1.0,2,10,workout_1
2,Lateral Walks,1.0,3,1,workout_1
3,Banded Goblet Squats,1.0,3,10,workout_1
4,Goblet Lunges,1.0,3,10,workout_1
...,...,...,...,...,...
1272,Plank Circuit,1.0,4,8,workout_123
1273,SL Glute Bridges,1.0,3,10,workout_123
1274,Mountain Climbers*,1.0,3,30,workout_123
1275,Burpees*,1.0,3,10,workout_123


In [107]:
dfs=[]
for client in all_formatted_data:
    df=pd.DataFrame(all_formatted_data[client])
    df['name']=client
    dfs.append(df)

df=pd.concat(dfs, axis=0)

In [110]:
df.sort_values(by=['name', 'workout_number'], inplace=True)
df=df.reset_index(drop=True)
df

,exercise,weight,sets,reps,workout_number,name
0,J-Jacks,1.0,1.0,50.0,workout_1,Abelev
1,Foam Roll,1.0,1.0,1.0,workout_1,Abelev
2,Lb Warm Up,1.0,2.0,1.0,workout_1,Abelev
3,UpWard/Down Dog,1.0,2.0,1.0,workout_1,Abelev
4,"Mountain Climbers (30,15/15, 15/15)",1.0,3.0,30.0,workout_1,Abelev
...,...,...,...,...,...,...
16528,SL RDL,1,3.0,8.0,workout_99,Yelena
16529,Prone Incline OH Press,1,3.0,10.0,workout_99,Yelena
16530,SA Renegade Row,1,3.0,10.0,workout_99,Yelena
16531,"Standing I, Y, T's",1,3.0,10.0,workout_99,Yelena


In [111]:
db_workouts

,name,workout_id,exercise,sets,reps,weight
0,Abelev,60,Banded Squats,3,10,1
1,Abelev,60,RFE Lunges,3,10,1
2,Abelev,60,Side Planks,3,30,1
3,Abelev,60,Banded Walks,3,60,1
4,Abelev,60,Split Squat Jumps,3,10,1
...,...,...,...,...,...,...
1872,Yelena,306,RDL,4,5,65
1873,Yelena,306,BB Hip Thurst,4,5,65
1874,Yelena,306,Split Squat Jumps,3,5,1
1875,Yelena,306,Mountain Climbers,3,30,1


In [114]:
df['workout_id']=df['workout_number']
df.drop(columns=['workout_number'], inplace=True)
df=df[['name', 'workout_id', 'exercise', 'sets', 'reps', 'weight']]


In [136]:
df['sets'] = pd.to_numeric(df['sets'], errors='coerce')
df['reps'] = pd.to_numeric(df['reps'], errors='coerce')
df['weight'] = pd.to_numeric(df['weight'], errors='coerce')

# fill NaN with default value, here I am using 0 as an example
df.dropna(0, inplace=True)

# Now you can convert to integer safely
df['sets'] = df['sets'].astype(int)
df['reps'] = df['reps'].astype(int)
df['weight'] = df['weight'].astype(int)


/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_17972/1478074816.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  df.dropna(0, inplace=True)


In [137]:
df

,name,workout_id,exercise,sets,reps,weight
0,Abelev,workout_1,J-Jacks,1,50,1
1,Abelev,workout_1,Foam Roll,1,1,1
2,Abelev,workout_1,Lb Warm Up,2,1,1
3,Abelev,workout_1,UpWard/Down Dog,2,1,1
4,Abelev,workout_1,"Mountain Climbers (30,15/15, 15/15)",3,30,1
...,...,...,...,...,...,...
16528,Yelena,workout_99,SL RDL,3,8,1
16529,Yelena,workout_99,Prone Incline OH Press,3,10,1
16530,Yelena,workout_99,SA Renegade Row,3,10,1
16531,Yelena,workout_99,"Standing I, Y, T's",3,10,1


In [168]:
all_workouts=pd.concat([df, db_workouts], axis=0)
all_workouts.sort_values(by=['name', 'workout_id'], inplace=True)
all_workouts.reset_index(drop=True, inplace=True)
all_workouts['exercise']=all_workouts['exercise'].str.lower()
all_workouts

,name,workout_id,exercise,sets,reps,weight
0,Abelev,60,banded squats,3,10,1
1,Abelev,60,rfe lunges,3,10,1
2,Abelev,60,side planks,3,30,1
3,Abelev,60,banded walks,3,60,1
4,Abelev,60,split squat jumps,3,10,1
...,...,...,...,...,...,...
18357,Yelena,workout_99,sl rdl,3,8,1
18358,Yelena,workout_99,prone incline oh press,3,10,1
18359,Yelena,workout_99,sa renegade row,3,10,1
18360,Yelena,workout_99,"standing i, y, t's",3,10,1


### Determine the most common focus for a single user's workouts. Apply approach to all subjects after

In [169]:
test=all_workouts[all_workouts['name']=='Misha']
test

,name,workout_id,exercise,sets,reps,weight
7919,Misha,41,offset squats,3,10,35
7920,Misha,41,squat jumps,3,10,1
7921,Misha,41,sa oh static lunge,3,10,15
7922,Misha,41,sl glute bridges,3,10,1
7923,Misha,41,yoga ball hamstring curls,3,10,1
...,...,...,...,...,...,...
11360,Misha,workout_99,plank to pushup,3,30,1
11361,Misha,workout_99,forward lunges,3,15,1
11362,Misha,workout_99,rfe lunge,3,15,1
11363,Misha,workout_99,bicycles,3,30,1


In [287]:
categories = {
    'upper_body_pulls': {
        'include': ['pull', 'row', 'down', 'point', 'renegade', 'chin', 'curl', 'snow', 'raise','prone', 'i,y,t'],
        'exclude': ['leg', 'yoga', 'hamstring', 'plank', 'calf', 'leg extension', 'dog', 'leg abduction', 'led adduction', 'oppo arm'],
    },
    'upper_body_pushes': {
        'include': ['bench', 'push', 'press', 'oh', 'incline', 'triceps', 'shoulders'],
        'exclude': ['lunge', 'squat', 'row', "i'y't", 'i, y, t','leg', 'prone', 'i,y,t', "t'"],
    },
    'cardio': {
        'include':['stairs', 'mountain', 'burp', 'sprint', 'rower', 'bike', 'treadmill'],
        'exclude': [],
    },
    'core': { 
        'include': ['plank', 'bug', 'banded', 'hold', '3:45', 'bicyle', 'pallof', 'twist', 'rotation', 'walks', 'crunch', 'situp', 'squeeze and reach', 'in and out', 'hyperextensions', 'passes'],
        'exclude': ['squat', 'lunge', 'standing', 'burpees'],
    },
    'plyos': {
        'include': ['hop', 'jump', 'land'],
        'exclude': [],
    },
    'lower_body_pushes': {
        'include': ['squat', 'lunge', 'split', 'step', 'leg press', 'sit', 'leg abduction'],
        'exclude': ['jump'],
    },
    'lower_body_pulls': {
        'include': ['dl', 'deadlift', '\\bthrust\\b', 'thurst', 'bridge', 'hamstring', 'swings', 'hypers', 'nordic', 'leg adduction'],
        'exclude': ['jump'],
    },
    'warm_up/cool_down': {
        'include': ['jacks', 'cat cow', 'clam', 'banded abductions', 'stretch', 'frog', 'pigeon', 'foam roll', 'dog', 'oppo'],
        'exclude': [],
    },
}


# for category, keywords in categories.items():
#     include_pattern = '|'.join(keywords['include'])
#     if keywords['exclude']:
#         exclude_pattern = ''.join(f"(?!.*{word})" for word in keywords['exclude'])  # Chain negative lookaheads
#         regex_pattern = f"{exclude_pattern}.*{include_pattern}"
#     else:
#         regex_pattern = include_pattern
    
#     print(f"Category: {category}")
#     print(f"Regex: {regex_pattern}")
#     matching_rows = test['exercise'].str.contains(regex_pattern, case=False)
#     print(f"Matching Rows: {matching_rows.sum()}")
#     test.loc[matching_rows, 'category'] = category
#     print("\n")




In [204]:
test_grouped =test.groupby(['name', 'workout_id'])['category'].agg(lambda x: x.value_counts().index[0])
test_grouped

name   workout_id
Misha  41            lower_body_pushes
       42            lower_body_pushes
       54                         core
       55                       cardio
       56                       cardio
                           ...        
       workout_95                 core
       workout_96                 core
       workout_97                 core
       workout_98                 core
       workout_99                 core
Name: category, Length: 371, dtype: object

In [262]:
def map_to_push_pull(category):
    if 'push' in category:
        return 'pushes'
    elif 'pull' in category:
        return 'pulls'
    else:
        return category

test['higher_category'] = test['category'].apply(map_to_push_pull)

test_grouped = test.groupby(['name', 'workout_id'])['higher_category'].agg(lambda x: x.value_counts().index[0]).reset_index()


KeyError: 'category'

In [207]:
test_grouped['higher_category'].value_counts()

core      131
pulls     118
pushes    112
cardio     10
Name: higher_category, dtype: int64

In [209]:
test

,name,workout_id,exercise,sets,reps,weight,category,higher_category
7919,Misha,41,offset squats,3,10,35,lower_body_pushes,pushes
7920,Misha,41,squat jumps,3,10,1,lower_body_pushes,pushes
7921,Misha,41,sa oh static lunge,3,10,15,lower_body_pushes,pushes
7922,Misha,41,sl glute bridges,3,10,1,lower_body_pulls,pulls
7923,Misha,41,yoga ball hamstring curls,3,10,1,lower_body_pulls,pulls
...,...,...,...,...,...,...,...,...
11360,Misha,workout_99,plank to pushup,3,30,1,core,core
11361,Misha,workout_99,forward lunges,3,15,1,lower_body_pushes,pushes
11362,Misha,workout_99,rfe lunge,3,15,1,lower_body_pushes,pushes
11363,Misha,workout_99,bicycles,3,30,1,core,core


In [210]:
def map_to_upper_lower(category):
    if 'upper' in category:
        return 'upper'
    elif 'lower' in category:
        return 'lower'
    else:
        return category
    
test['upper_lower'] = test['category'].apply(map_to_upper_lower)
test_grouped = test.groupby(['name', 'workout_id'])['upper_lower'].agg(lambda x: x.value_counts().index[0]).reset_index()
test_grouped['upper_lower'].value_counts()

/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_17972/2119106706.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['upper_lower'] = test['category'].apply(map_to_upper_lower)


core      144
upper     127
lower      89
cardio     11
Name: upper_lower, dtype: int64

In [214]:
test[test['workout_id']==42]

,name,workout_id,exercise,sets,reps,weight,category,higher_category,upper_lower
7926,Misha,42,sa db swings,3,10,35,core,core,core
7927,Misha,42,split squat jumps,3,5,1,lower_body_pushes,pushes,lower
7928,Misha,42,static lunges,3,10,35,lower_body_pushes,pushes,lower
7929,Misha,42,sa oh walks,3,30,35,core,core,core
7930,Misha,42,sa farmers walks,3,30,35,core,core,core
7931,Misha,42,rfe lunge holds,3,10,1,lower_body_pushes,pushes,lower
7932,Misha,42,db sl rdl,3,6,35,lower_body_pulls,pulls,lower
7933,Misha,42,sa db oh press,3,10,35,upper_body_pushes,pushes,upper
7934,Misha,42,rfe lunges,3,10,1,lower_body_pushes,pushes,lower
7935,Misha,42,hill sprints,8,1,1,cardio,cardio,cardio


In [215]:
test_grouped = test.groupby(['name', 'workout_id'])[['upper_lower', 'higher_category']].agg(lambda x: x.value_counts().index[0]).reset_index()
test_grouped


,name,workout_id,upper_lower,higher_category
0,Misha,41,lower,pulls
1,Misha,42,lower,pushes
2,Misha,54,core,core
3,Misha,55,cardio,cardio
4,Misha,56,cardio,cardio
...,...,...,...,...
366,Misha,workout_95,core,pushes
367,Misha,workout_96,upper,core
368,Misha,workout_97,core,core
369,Misha,workout_98,upper,core


In [216]:
test[test['workout_id']=='workout_95']

,name,workout_id,exercise,sets,reps,weight,category,higher_category,upper_lower
11321,Misha,workout_95,mountain climbers*,3,30,1,cardio,cardio,cardio
11322,Misha,workout_95,side planks*,3,30,1,core,core,core
11323,Misha,workout_95,trx pushups,3,15,1,upper_body_pushes,pushes,upper
11324,Misha,workout_95,pushups,3,15,1,upper_body_pushes,pushes,upper
11325,Misha,workout_95,plank to pushup,3,30,1,core,core,core
11326,Misha,workout_95,forward lunges,3,15,1,lower_body_pushes,pushes,lower
11327,Misha,workout_95,rfe lunge,3,15,1,lower_body_pushes,pushes,lower
11328,Misha,workout_95,bicycles,3,30,1,core,core,core
11329,Misha,workout_95,burpees,3,15,1,cardio,cardio,cardio


We are getting closer and closer to being able to map the workouts to what they are but in the example above we can see that this would be mapped to core because of three core exercises despite this likley being mapped to a full body (ton of subjectivity here, but I would map it to full body)

In [217]:
def aggregate_categories(categories):
    count_upper = sum(category == 'upper' for category in categories)
    count_lower = sum(category == 'lower' for category in categories)
    count_core = sum(category == 'core' for category in categories)

    if count_upper + count_lower > count_core:
        return 'full_body'
    elif count_upper > count_lower:
        return 'upper'
    elif count_lower > count_upper:
        return 'lower'
    else:
        return 'core'

test_grouped = test.groupby(['name', 'workout_id'])['upper_lower'].agg(aggregate_categories).reset_index()
test_grouped

,name,workout_id,upper_lower
0,Misha,41,full_body
1,Misha,42,full_body
2,Misha,54,core
3,Misha,55,core
4,Misha,56,core
...,...,...,...
366,Misha,workout_95,full_body
367,Misha,workout_96,full_body
368,Misha,workout_97,core
369,Misha,workout_98,full_body


In [218]:
test_grouped['upper_lower'].value_counts()

full_body    297
lower         36
upper         29
core           9
Name: upper_lower, dtype: int64

In [220]:
test['exercise_category']=test['category']
test.drop(columns=['category'], inplace=True)
test['push_pull']=test['higher_category']
test.drop(columns=['higher_category'], inplace=True)
test['movement_category']=test['exercise_category']
test.drop(columns=['exercise_category'], inplace=True)

/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_17972/419677375.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['exercise_category']=test['category']
/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_17972/419677375.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(columns=['category'], inplace=True)
/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_17972/419677375.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

,name,workout_id,exercise,sets,reps,weight,upper_lower,exercise_category,push_pull
7919,Misha,41,offset squats,3,10,35,lower,lower_body_pushes,pushes
7920,Misha,41,squat jumps,3,10,1,lower,lower_body_pushes,pushes
7921,Misha,41,sa oh static lunge,3,10,15,lower,lower_body_pushes,pushes
7922,Misha,41,sl glute bridges,3,10,1,lower,lower_body_pulls,pulls
7923,Misha,41,yoga ball hamstring curls,3,10,1,lower,lower_body_pulls,pulls
...,...,...,...,...,...,...,...,...,...
11360,Misha,workout_99,plank to pushup,3,30,1,core,core,core
11361,Misha,workout_99,forward lunges,3,15,1,lower,lower_body_pushes,pushes
11362,Misha,workout_99,rfe lunge,3,15,1,lower,lower_body_pushes,pushes
11363,Misha,workout_99,bicycles,3,30,1,core,core,core


In [221]:
test

/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_17972/848098545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['movement_category']=test['exercise_category']
/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_17972/848098545.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(columns=['exercise_category'], inplace=True)


,name,workout_id,exercise,sets,reps,weight,upper_lower,push_pull,movement_category
7919,Misha,41,offset squats,3,10,35,lower,pushes,lower_body_pushes
7920,Misha,41,squat jumps,3,10,1,lower,pushes,lower_body_pushes
7921,Misha,41,sa oh static lunge,3,10,15,lower,pushes,lower_body_pushes
7922,Misha,41,sl glute bridges,3,10,1,lower,pulls,lower_body_pulls
7923,Misha,41,yoga ball hamstring curls,3,10,1,lower,pulls,lower_body_pulls
...,...,...,...,...,...,...,...,...,...
11360,Misha,workout_99,plank to pushup,3,30,1,core,core,core
11361,Misha,workout_99,forward lunges,3,15,1,lower,pushes,lower_body_pushes
11362,Misha,workout_99,rfe lunge,3,15,1,lower,pushes,lower_body_pushes
11363,Misha,workout_99,bicycles,3,30,1,core,core,core


### Now with these categories we can analyzie the specific exercise movements, general workout_movements, and the body segment being targeted.

Let's apply this to the whole dataframe

Make sure to drop any NaN

In [288]:
for category, keywords in categories.items():
    include_pattern = '|'.join(keywords['include'])
    if keywords['exclude']:
        exclude_pattern = ''.join(f"(?!.*{word})" for word in keywords['exclude'])  # Chain negative lookaheads
        regex_pattern = f"{exclude_pattern}.*{include_pattern}"
    else:
        regex_pattern = include_pattern
    
    print(f"Category: {category}")
    print(f"Regex: {regex_pattern}")
    matching_rows = all_workouts['exercise'].str.contains(regex_pattern, case=False)
    if matching_rows.isnull().any():
        print(f"NaN values found in matching rows for category {category} and regex {regex_pattern}")
    print(f"Matching Rows: {matching_rows.sum()}")
    all_workouts.loc[matching_rows, 'category'] = category
    print("\n")

Category: upper_body_pulls
Regex: (?!.*leg)(?!.*yoga)(?!.*hamstring)(?!.*plank)(?!.*calf)(?!.*leg extension)(?!.*dog)(?!.*leg abduction)(?!.*led adduction)(?!.*oppo arm).*pull|row|down|point|renegade|chin|curl|snow|raise|prone|i,y,t
Matching Rows: 3545


Category: upper_body_pushes
Regex: (?!.*lunge)(?!.*squat)(?!.*row)(?!.*i'y't)(?!.*i, y, t)(?!.*leg)(?!.*prone)(?!.*i,y,t)(?!.*t').*bench|push|press|oh|incline|triceps|shoulders
Matching Rows: 3087


Category: cardio
Regex: stairs|mountain|burp|sprint|rower|bike|treadmill
Matching Rows: 1880


Category: core
Regex: (?!.*squat)(?!.*lunge)(?!.*standing)(?!.*burpees).*plank|bug|banded|hold|3:45|bicyle|pallof|twist|rotation|walks|crunch|situp|squeeze and reach|in and out|hyperextensions|passes
Matching Rows: 3557


Category: plyos
Regex: hop|jump|land
Matching Rows: 775


Category: lower_body_pushes
Regex: (?!.*jump).*squat|lunge|split|step|leg press|sit|leg abduction
Matching Rows: 2836


Category: lower_body_pulls
Regex: (?!.*jump).*dl|de

In [289]:
all_workouts=all_workouts.dropna(subset=['category'])
all_workouts['exercise_movement']=all_workouts['category']
all_workouts.drop(columns=['category'], inplace=True)

In [290]:
all_workouts['upper_lower'] = all_workouts['exercise_movement'].apply(map_to_upper_lower)
all_workouts['push_pull'] = all_workouts['exercise_movement'].apply(map_to_push_pull)
all_workouts

,name,workout_id,exercise,sets,reps,weight,upper_lower,push_pull,exercise_movement
0,Abelev,60,banded squats,3,10,1,lower,pushes,lower_body_pushes
1,Abelev,60,rfe lunges,3,10,1,lower,pushes,lower_body_pushes
2,Abelev,60,side planks,3,30,1,core,core,core
3,Abelev,60,banded walks,3,60,1,core,core,core
4,Abelev,60,split squat jumps,3,10,1,lower,pushes,lower_body_pushes
...,...,...,...,...,...,...,...,...,...
18356,Yelena,workout_99,1-point sa db row,3,10,1,upper,pulls,upper_body_pulls
18357,Yelena,workout_99,sl rdl,3,8,1,lower,pulls,lower_body_pulls
18358,Yelena,workout_99,prone incline oh press,3,10,1,upper,pushes,upper_body_pushes
18359,Yelena,workout_99,sa renegade row,3,10,1,upper,pulls,upper_body_pulls


### What type of movements do I prescribe the most

1. Core= Brace, twist, rotation, anti-rotation etc
2. Push= Push, press, etc
3. Lower Body Pull= Hinge, deadlift, bridges 
4. upper body pull= pull, row, etc
5. lower bodu push = squat, lunge etc

In [291]:
all_workouts['exercise_movement'].value_counts()

core                 3013
lower_body_pushes    2657
warm_up/cool_down    2496
upper_body_pushes    2104
upper_body_pulls     1960
lower_body_pulls     1860
cardio               1784
plyos                 322
Name: exercise_movement, dtype: int64

### Do I split more by upper/lower or full body?

In [292]:

all_groups=all_workouts.groupby(['name', 'workout_id'])['upper_lower'].agg(aggregate_categories).reset_index()
all_groups['upper_lower'].value_counts()

full_body    1695
lower         158
core           93
upper          55
Name: upper_lower, dtype: int64

### Do I gear more towards upper body exercises or lower body exercises?

In [293]:
all_workouts['upper_lower'].value_counts()

lower                4517
upper                4064
core                 3013
warm_up/cool_down    2496
cardio               1784
plyos                 322
Name: upper_lower, dtype: int64

### Does the most popular exercise prescribed align with teh previous results?

In [305]:
counts = all_workouts['exercise'].value_counts()
popular=counts[counts>100]


In [307]:
popular #Pretty comparable, there is actually more data here than in the previous analysis

mountain climbers      731
burpees                436
j-jacks                333
banded walks           331
side planks            300
upward/downward dog    292
plank to pushup        280
pushups                255
sl rdl                 248
3:45 circuit           240
split squat jumps      224
foam roll              205
plank circuit          190
sa db row              174
db bench press         170
dead bugs              165
squat jumps            162
sl glute bridges       162
sa renegade row        151
db oh press            140
prone oh press         139
squat thrusters        136
up/down dog            127
rdl                    119
rfe lunge              119
rfe lunges             117
hamstring and calf     117
hollow holds           115
mountain climbers*     108
stretch                102
sl rdl row             101
Name: exercise, dtype: int64

### I am a little surprised that upper body pulls is the second highest exercise category simply due to how many workouts I have done from home and bodywieght where those are generaly harder to do, so I am going to audit it and see if any were erroneosuly added

In [285]:
pull_exercises=all_workouts[all_workouts['exercise_movement']=='upper_body_pulls']
pull_exercises['exercise'].unique()

array(['sa renegade row', "i,y,t's", 'sa db row', 'seal rows',
       'prone snow angels', 'upward/down dog', '1-point sa db row',
       '2-point sa db row', 'snow angels', 'upward/downward dog',
       'bilateral bent over row', "standing i,y,t's", 'db pullover',
       'up/down & 90/90', 'standing snow angels', 'stansing snow angels',
       'upward dog/down dog w/ankle emphasis',
       'bent over row-horizonatal pull', 'reneagde row',
       'alternating renegade row-vertical pull',
       'lu raises-vertical pull', 'upward downward dog', 'up/down dog',
       'block calf raises', 'upward dog/down dog', 'db pullover ',
       'curls/tricep kickbacks', 'sl calf raises', 'lying db pullover',
       'renegade row', 'db pullover**', 'sa db row**',
       'standing snow angel**', 'db pullover*', 'sa renegade row**',
       'bilateral db pullover ', 'prone db snow angels', 'pullover',
       'db bent over row', 'bear crawl oppo arm/leg raise',
       'oppo arm/leg raise', '2-point reneg

A few here or there but generally it looks like it is correct. I am removing upward/downward dog and the machines from the list


### After removing just upward/downward dog and leg extensions it reduced it by alomst 800 putting it on par with lower body pulls which makes more sense

## Export This data for Tableau

In [308]:
all_workouts.to_csv('all_workouts.csv', index=False)